# YogaPartner

Our datasets are in
- `.\data\v1`
- `.\data\v2`

Our frontend code is in `.\android`

Our generated data goes in `.\intermediates`

In [1]:
import os
import matplotlib.pyplot as plt
from pathlib import Path

version = 2

## Global functions

In [2]:
def getPath():
    return r".\data\v" + str(version)

def getClasses():
    path = getPath()
    return os.listdir(path)

def getPathOfClass(className):
    path = getPath()
    return os.path.join(path, className)

## Dataset stats

In [6]:
def visualizeDataSet():
    classFolders = getClasses()
    counts = []
    imageTypes = {}
    for classFolder in classFolders:
        images = os.listdir(getPathOfClass(classFolder))
        count = len(images)
        counts.append(count)
        for image in images:
            image_path = os.path.join(getPathOfClass(classFolder), image)
            p = Path(image_path)
            ext = p.suffix
            imageTypes[ext] = imageTypes.get(ext, 0) + 1

    print("Dataset frequency distribution")
    for i in range(len(classFolders)):
        className = classFolders[i]
        count = counts[i]
        print(className + " : " + str(count))

    print()

    print("Image type frequency distribution")
    for key in imageTypes.keys():
        count = imageTypes[key]
        print(key + " : " + str(count))

    print()
    
visualizeDataSet()

Dataset frequency distribution
adho_mukha_shvanasana : 435
bhujangasana : 667
bidalasana : 315
phalakasana : 145
ustrasana : 66
utkatasana : 226
utkata_konasana : 209
vasisthasana : 148
virabhadrasana_i : 221
virabhadrasana_ii : 389
vrikshasana : 318

Image type frequency distribution
.jpg : 3096
.jpeg : 43



## Cleaning Data

### Convert to Jpeg

In [13]:
from PIL import Image
import os
import re
from pathlib import Path

classFolders = getClasses()
for className in classFolders:
    class_path = getPathOfClass(className)
    files = os.listdir(class_path)
    for f in files:
        old_path = os.path.join(class_path, f)
        p = Path(old_path)
        ext = p.suffix
        if ext == ".webp":
            print("Converting " + f)
            new_file_name = f"{p.stem}.jepg"
            im = Image.open(old_path).convert("RGB")
            new_file_path = os.path.join(class_path, new_file_name)
            print(new_file_path)
            im.save(new_file_path, "jepg")
            print("Removing " + f)
            os.remove(old_path)
        if ext == ".png":
            print("Converting " + f)
            new_file_name = f"{p.stem}.jpg"
            im = Image.open(old_path).convert("RGB")
            new_file_path = os.path.join(class_path, new_file_name)
            print(new_file_path)
            im.save(new_file_path)
            print("Removing " + f)
            os.remove(old_path)
        if ext == ".JPG":
            print("Converting " + f)
            new_file_name = f"{p.stem}.jpg"
            im = Image.open(old_path).convert("RGB")
            new_file_path = os.path.join(class_path, new_file_name)
            print(new_file_path)
            im.save(new_file_path)
            print("Removing " + f)
            os.remove(old_path)
print("Converted Webp and PNG")

Converted Webp and PNG


### Finding duplicate images

In [14]:
from difPy import dif
import tqdm

classFolders = getClasses()
for className in tqdm.tqdm(classFolders, position=0):
    class_path = getPathOfClass(className)
    search = dif(class_path, delete=True)
print("Completed duplicate image search")

  0%|                                                                                           | 0/11 [00:00<?, ?it/s]

Found 2 images with one or more duplicate/similar images in 36.9846 seconds.
Are you sure you want to delete all lower resolution duplicate images? 
This cannot be undone. (y/n)y


  9%|███████▌                                                                           | 1/11 [00:40<06:44, 40.42s/it]


Deleted file: .\data\v2\adho_mukha_shvanasana/adho_mukha_shvanasana_88.jpg
Deleted file: .\data\v2\adho_mukha_shvanasana/adho_mukha_shvanasana_1.jpg
***
Deleted 2 images.
Found 657 images with one or more duplicate/similar images in 322.4201 seconds.
Are you sure you want to delete all lower resolution duplicate images? 
This cannot be undone. (y/n)y


 18%|██████████████▉                                                                   | 2/11 [07:20<37:47, 251.98s/it]

...

 91%|██████████████████████████████████████████████████████████████████████████▌       | 10/11 [11:56<00:55, 55.81s/it]


Deleted file: .\data\v2\virabhadrasana_ii/virabhadrasana_ii_174.jpg
Deleted file: .\data\v2\virabhadrasana_ii/virabhadrasana_ii_334.jpg
Deleted file: .\data\v2\virabhadrasana_ii/virabhadrasana_ii_165.jpg
Deleted file: .\data\v2\virabhadrasana_ii/virabhadrasana_ii_367.jpg
Deleted file: .\data\v2\virabhadrasana_ii/virabhadrasana_ii_9.jpg
Deleted file: .\data\v2\virabhadrasana_ii/virabhadrasana_ii_196.jpg
Deleted file: .\data\v2\virabhadrasana_ii/virabhadrasana_ii_109.jpg
Deleted file: .\data\v2\virabhadrasana_ii/virabhadrasana_ii_274.jpg
Deleted file: .\data\v2\virabhadrasana_ii/virabhadrasana_ii_182.jpg
Deleted file: .\data\v2\virabhadrasana_ii/virabhadrasana_ii_290.jpg
Deleted file: .\data\v2\virabhadrasana_ii/virabhadrasana_ii_388.jpg
Deleted file: .\data\v2\virabhadrasana_ii/virabhadrasana_ii_219.jpg
Deleted file: .\data\v2\virabhadrasana_ii/virabhadrasana_ii_143.jpg
Deleted file: .\data\v2\virabhadrasana_ii/virabhadrasana_ii_23.jpg
Deleted file: .\data\v2\virabhadrasana_ii/virabhad

Found 310 images with one or more duplicate/similar images in 63.1503 seconds.
Are you sure you want to delete all lower resolution duplicate images? 
This cannot be undone. (y/n)y


100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [13:04<00:00, 71.34s/it]


Deleted file: .\data\v2\vrikshasana/vrikshasana_24.jpg
Deleted file: .\data\v2\vrikshasana/vrikshasana_315.jpg
Deleted file: .\data\v2\vrikshasana/vrikshasana_141.jpg
Deleted file: .\data\v2\vrikshasana/vrikshasana_286.jpg
Deleted file: .\data\v2\vrikshasana/vrikshasana_225.jpg
Deleted file: .\data\v2\vrikshasana/vrikshasana_257.jpg
Deleted file: .\data\v2\vrikshasana/vrikshasana_166.jpg
Deleted file: .\data\v2\vrikshasana/vrikshasana_279.jpg
Deleted file: .\data\v2\vrikshasana/vrikshasana_235.jpg
Deleted file: .\data\v2\vrikshasana/vrikshasana_312.jpg
Deleted file: .\data\v2\vrikshasana/vrikshasana_34.jpg
Deleted file: .\data\v2\vrikshasana/vrikshasana_262.jpg
Deleted file: .\data\v2\vrikshasana/vrikshasana_13.jpg
Deleted file: .\data\v2\vrikshasana/vrikshasana_317.jpg
Deleted file: .\data\v2\vrikshasana/vrikshasana_153.jpg
Deleted file: .\data\v2\vrikshasana/vrikshasana_219.jpg
Deleted file: .\data\v2\vrikshasana/vrikshasana_237.jpg
Deleted file: .\data\v2\vrikshasana/vrikshasana_27

### Cleaning up folder names

In [15]:
import os
import re
from pathlib import Path

classFolders = getClasses()
for className in classFolders:
    old_path = getPathOfClass(className)
    new_name = re.sub(r"\s+", "_", className)
    if new_name != className:
        print("Renaming " + className)
    new_path = os.path.join(getPath(), new_name)
    os.rename(old_path, new_path)
print("Renamed class names")

Renamed class names


### Removing non supported images

In [16]:
from PIL import Image
import os
import re
from pathlib import Path

classFolders = getClasses()
for className in classFolders:
    class_path = getPathOfClass(className)
    files = os.listdir(class_path)
    count = 1
    for f in files:
        f_path = os.path.join(class_path, f)
        if not (f.endswith(".jpg") or f.endswith(".jpeg")):
            os.remove(f_path)
            print("Removed " + f)
print("Finished removing non suporrted images")

Finished removing non suporrted images


### Cleaning up file names

In [22]:
from PIL import Image
import os
import shutil
import re
from pathlib import Path

classFolders = getClasses()
for className in classFolders:
    class_path = getPathOfClass(className)
    files = os.listdir(class_path)
    count = 1
    for f in files:
        old_path = os.path.join(class_path, f)
        p = Path(old_path)
        new_file_name = f"{className}_{count}{p.suffix}"
        if new_file_name != f:
            print("Renaming " + f)
        new_file_path = os.path.join(class_path, new_file_name)
        if(os.path.exists(new_file_path)):
            print("Clash in names, change to unique pattern")
            break
        os.rename(old_path, new_file_path)
        count += 1
        
print("Finished cleaning file names")

Renaming adho_mukha_shvanasana_sharkaboi1.jpg
Renaming adho_mukha_shvanasana_sharkaboi10.jpg
Renaming adho_mukha_shvanasana_sharkaboi100.jpg
Renaming adho_mukha_shvanasana_sharkaboi101.jpg
Renaming adho_mukha_shvanasana_sharkaboi102.jpg
Renaming adho_mukha_shvanasana_sharkaboi103.jpg
Renaming adho_mukha_shvanasana_sharkaboi104.jpg
Renaming adho_mukha_shvanasana_sharkaboi105.jpg
Renaming adho_mukha_shvanasana_sharkaboi106.jpg
Renaming adho_mukha_shvanasana_sharkaboi107.jpg
Renaming adho_mukha_shvanasana_sharkaboi108.jpg
Renaming adho_mukha_shvanasana_sharkaboi109.jpg
Renaming adho_mukha_shvanasana_sharkaboi11.jpg
Renaming adho_mukha_shvanasana_sharkaboi110.jpg
Renaming adho_mukha_shvanasana_sharkaboi111.jpg
Renaming adho_mukha_shvanasana_sharkaboi112.jpg
Renaming adho_mukha_shvanasana_sharkaboi113.jpg
Renaming adho_mukha_shvanasana_sharkaboi114.jpg
Renaming adho_mukha_shvanasana_sharkaboi115.jpg
Renaming adho_mukha_shvanasana_sharkaboi116.jpg
Renaming adho_mukha_shvanasana_sharkaboi117.

Renaming adho_mukha_shvanasana_sharkaboi384.jpg
Renaming adho_mukha_shvanasana_sharkaboi385.jpg
Renaming adho_mukha_shvanasana_sharkaboi386.jpg
Renaming adho_mukha_shvanasana_sharkaboi387.jpg
Renaming adho_mukha_shvanasana_sharkaboi388.jpg
Renaming adho_mukha_shvanasana_sharkaboi389.jpg
Renaming adho_mukha_shvanasana_sharkaboi39.jpg
Renaming adho_mukha_shvanasana_sharkaboi390.jpg
Renaming adho_mukha_shvanasana_sharkaboi391.jpg
Renaming adho_mukha_shvanasana_sharkaboi392.jpg
Renaming adho_mukha_shvanasana_sharkaboi393.jpg
Renaming adho_mukha_shvanasana_sharkaboi394.jpg
Renaming adho_mukha_shvanasana_sharkaboi395.jpg
Renaming adho_mukha_shvanasana_sharkaboi396.jpg
Renaming adho_mukha_shvanasana_sharkaboi397.jpg
Renaming adho_mukha_shvanasana_sharkaboi398.jpg
Renaming adho_mukha_shvanasana_sharkaboi399.jpg
Renaming adho_mukha_shvanasana_sharkaboi4.jpg
Renaming adho_mukha_shvanasana_sharkaboi40.jpg
Renaming adho_mukha_shvanasana_sharkaboi400.jpg
Renaming adho_mukha_shvanasana_sharkaboi401.

Renaming bhujangasana_sharkaboi27.jpg
Renaming bhujangasana_sharkaboi270.jpg
Renaming bhujangasana_sharkaboi271.jpg
Renaming bhujangasana_sharkaboi272.jpg
Renaming bhujangasana_sharkaboi273.jpg
Renaming bhujangasana_sharkaboi274.jpg
Renaming bhujangasana_sharkaboi275.jpg
Renaming bhujangasana_sharkaboi276.jpg
Renaming bhujangasana_sharkaboi277.jpg
Renaming bhujangasana_sharkaboi278.jpg
Renaming bhujangasana_sharkaboi279.jpg
Renaming bhujangasana_sharkaboi28.jpg
Renaming bhujangasana_sharkaboi280.jpg
Renaming bhujangasana_sharkaboi281.jpg
Renaming bhujangasana_sharkaboi282.jpg
Renaming bhujangasana_sharkaboi283.jpg
Renaming bhujangasana_sharkaboi284.jpg
Renaming bhujangasana_sharkaboi285.jpg
Renaming bhujangasana_sharkaboi286.jpg
Renaming bhujangasana_sharkaboi287.jpg
Renaming bhujangasana_sharkaboi288.jpg
Renaming bhujangasana_sharkaboi289.jpg
Renaming bhujangasana_sharkaboi29.jpg
Renaming bhujangasana_sharkaboi290.jpg
Renaming bhujangasana_sharkaboi291.jpg
Renaming bhujangasana_sharka

Renaming bhujangasana_sharkaboi579.jpg
Renaming bhujangasana_sharkaboi58.jpg
Renaming bhujangasana_sharkaboi580.jpg
Renaming bhujangasana_sharkaboi581.jpg
Renaming bhujangasana_sharkaboi582.jpg
Renaming bhujangasana_sharkaboi583.jpg
Renaming bhujangasana_sharkaboi584.jpg
Renaming bhujangasana_sharkaboi585.jpg
Renaming bhujangasana_sharkaboi586.jpg
Renaming bhujangasana_sharkaboi587.jpg
Renaming bhujangasana_sharkaboi588.jpg
Renaming bhujangasana_sharkaboi589.jpg
Renaming bhujangasana_sharkaboi59.jpg
Renaming bhujangasana_sharkaboi590.jpg
Renaming bhujangasana_sharkaboi591.jpg
Renaming bhujangasana_sharkaboi592.jpg
Renaming bhujangasana_sharkaboi593.jpg
Renaming bhujangasana_sharkaboi594.jpg
Renaming bhujangasana_sharkaboi595.jpg
Renaming bhujangasana_sharkaboi596.jpg
Renaming bhujangasana_sharkaboi597.jpg
Renaming bhujangasana_sharkaboi598.jpg
Renaming bhujangasana_sharkaboi599.jpg
Renaming bhujangasana_sharkaboi6.jpg
Renaming bhujangasana_sharkaboi60.jpg
Renaming bhujangasana_sharkabo

Renaming bidalasana_sharkaboi175.jpg
Renaming bidalasana_sharkaboi176.jpg
Renaming bidalasana_sharkaboi177.jpg
Renaming bidalasana_sharkaboi178.jpg
Renaming bidalasana_sharkaboi179.jpg
Renaming bidalasana_sharkaboi18.jpg
Renaming bidalasana_sharkaboi180.jpeg
Renaming bidalasana_sharkaboi181.jpg
Renaming bidalasana_sharkaboi182.jpg
Renaming bidalasana_sharkaboi183.jpg
Renaming bidalasana_sharkaboi184.jpg
Renaming bidalasana_sharkaboi185.jpg
Renaming bidalasana_sharkaboi186.jpg
Renaming bidalasana_sharkaboi187.jpg
Renaming bidalasana_sharkaboi188.jpg
Renaming bidalasana_sharkaboi189.jpeg
Renaming bidalasana_sharkaboi19.jpg
Renaming bidalasana_sharkaboi190.jpg
Renaming bidalasana_sharkaboi191.jpg
Renaming bidalasana_sharkaboi192.jpg
Renaming bidalasana_sharkaboi193.jpg
Renaming bidalasana_sharkaboi194.jpg
Renaming bidalasana_sharkaboi195.jpg
Renaming bidalasana_sharkaboi196.jpg
Renaming bidalasana_sharkaboi197.jpg
Renaming bidalasana_sharkaboi198.jpg
Renaming bidalasana_sharkaboi199.jpg
R

...

Renaming utkata_konasana_sharkaboi151.jpeg
Renaming utkata_konasana_sharkaboi152.jpg
Renaming utkata_konasana_sharkaboi153.jpg
Renaming utkata_konasana_sharkaboi154.jpg
Renaming utkata_konasana_sharkaboi155.jpg
Renaming utkata_konasana_sharkaboi156.jpg
Renaming utkata_konasana_sharkaboi157.jpg
Renaming utkata_konasana_sharkaboi158.jpg
Renaming utkata_konasana_sharkaboi159.jpg
Renaming utkata_konasana_sharkaboi16.jpg
Renaming utkata_konasana_sharkaboi160.jpeg
Renaming utkata_konasana_sharkaboi161.jpg
Renaming utkata_konasana_sharkaboi162.jpg
Renaming utkata_konasana_sharkaboi163.jpg
Renaming utkata_konasana_sharkaboi164.jpg
Renaming utkata_konasana_sharkaboi165.jpg
Renaming utkata_konasana_sharkaboi166.jpg
Renaming utkata_konasana_sharkaboi167.jpg
Renaming utkata_konasana_sharkaboi168.jpg
Renaming utkata_konasana_sharkaboi169.jpg
Renaming utkata_konasana_sharkaboi17.jpg
Renaming utkata_konasana_sharkaboi170.jpg
Renaming utkata_konasana_sharkaboi171.jpg
Renaming utkata_konasana_sharkaboi

Renaming vasisthasana_sharkaboi65.jpg
Renaming vasisthasana_sharkaboi66.jpg
Renaming vasisthasana_sharkaboi67.jpg
Renaming vasisthasana_sharkaboi68.jpg
Renaming vasisthasana_sharkaboi69.jpg
Renaming vasisthasana_sharkaboi7.jpg
Renaming vasisthasana_sharkaboi70.jpg
Renaming vasisthasana_sharkaboi71.jpg
Renaming vasisthasana_sharkaboi72.jpg
Renaming vasisthasana_sharkaboi73.jpg
Renaming vasisthasana_sharkaboi74.jpg
Renaming vasisthasana_sharkaboi75.jpg
Renaming vasisthasana_sharkaboi76.jpg
Renaming vasisthasana_sharkaboi77.jpg
Renaming vasisthasana_sharkaboi78.jpg
Renaming vasisthasana_sharkaboi79.jpg
Renaming vasisthasana_sharkaboi8.jpg
Renaming vasisthasana_sharkaboi80.jpg
Renaming vasisthasana_sharkaboi81.jpg
Renaming vasisthasana_sharkaboi82.jpg
Renaming vasisthasana_sharkaboi83.jpg
Renaming vasisthasana_sharkaboi84.jpg
Renaming vasisthasana_sharkaboi85.jpg
Renaming vasisthasana_sharkaboi86.jpg
Renaming vasisthasana_sharkaboi87.jpg
Renaming vasisthasana_sharkaboi88.jpg
Renaming vasis

Renaming virabhadrasana_ii_sharkaboi153.jpg
Renaming virabhadrasana_ii_sharkaboi154.jpg
Renaming virabhadrasana_ii_sharkaboi155.jpg
Renaming virabhadrasana_ii_sharkaboi156.jpg
Renaming virabhadrasana_ii_sharkaboi157.jpg
Renaming virabhadrasana_ii_sharkaboi158.jpg
Renaming virabhadrasana_ii_sharkaboi159.jpg
Renaming virabhadrasana_ii_sharkaboi16.jpg
Renaming virabhadrasana_ii_sharkaboi160.jpg
Renaming virabhadrasana_ii_sharkaboi161.jpg
Renaming virabhadrasana_ii_sharkaboi162.jpg
Renaming virabhadrasana_ii_sharkaboi163.jpg
Renaming virabhadrasana_ii_sharkaboi164.jpg
Renaming virabhadrasana_ii_sharkaboi165.jpg
Renaming virabhadrasana_ii_sharkaboi166.jpg
Renaming virabhadrasana_ii_sharkaboi167.jpg
Renaming virabhadrasana_ii_sharkaboi168.jpg
Renaming virabhadrasana_ii_sharkaboi169.jpg
Renaming virabhadrasana_ii_sharkaboi17.jpg
Renaming virabhadrasana_ii_sharkaboi170.jpg
Renaming virabhadrasana_ii_sharkaboi171.jpg
Renaming virabhadrasana_ii_sharkaboi172.jpg
Renaming virabhadrasana_ii_sharkab

Renaming virabhadrasana_ii_sharkaboi369.jpg
Renaming virabhadrasana_ii_sharkaboi37.jpg
Renaming virabhadrasana_ii_sharkaboi370.jpg
Renaming virabhadrasana_ii_sharkaboi371.jpg
Renaming virabhadrasana_ii_sharkaboi372.jpg
Renaming virabhadrasana_ii_sharkaboi373.jpg
Renaming virabhadrasana_ii_sharkaboi374.jpg
Renaming virabhadrasana_ii_sharkaboi375.jpg
Renaming virabhadrasana_ii_sharkaboi376.jpg
Renaming virabhadrasana_ii_sharkaboi377.jpg
Renaming virabhadrasana_ii_sharkaboi378.jpg
Renaming virabhadrasana_ii_sharkaboi379.jpg
Renaming virabhadrasana_ii_sharkaboi38.jpg
Renaming virabhadrasana_ii_sharkaboi380.jpg
Renaming virabhadrasana_ii_sharkaboi381.jpg
Renaming virabhadrasana_ii_sharkaboi382.jpg
Renaming virabhadrasana_ii_sharkaboi383.jpg
Renaming virabhadrasana_ii_sharkaboi384.jpg
Renaming virabhadrasana_ii_sharkaboi385.jpg
Renaming virabhadrasana_ii_sharkaboi386.jpg
Renaming virabhadrasana_ii_sharkaboi387.jpg
Renaming virabhadrasana_ii_sharkaboi388.jpg
Renaming virabhadrasana_ii_sharkab

Renaming vrikshasana_sharkaboi280.jpg
Renaming vrikshasana_sharkaboi281.jpg
Renaming vrikshasana_sharkaboi282.jpg
Renaming vrikshasana_sharkaboi283.jpg
Renaming vrikshasana_sharkaboi284.jpg
Renaming vrikshasana_sharkaboi285.jpg
Renaming vrikshasana_sharkaboi286.jpg
Renaming vrikshasana_sharkaboi287.jpg
Renaming vrikshasana_sharkaboi288.jpg
Renaming vrikshasana_sharkaboi289.jpg
Renaming vrikshasana_sharkaboi29.jpg
Renaming vrikshasana_sharkaboi290.jpg
Renaming vrikshasana_sharkaboi291.jpg
Renaming vrikshasana_sharkaboi292.jpg
Renaming vrikshasana_sharkaboi293.jpg
Renaming vrikshasana_sharkaboi294.jpg
Renaming vrikshasana_sharkaboi295.jpg
Renaming vrikshasana_sharkaboi296.jpg
Renaming vrikshasana_sharkaboi297.jpg
Renaming vrikshasana_sharkaboi298.jpg
Renaming vrikshasana_sharkaboi299.jpg
Renaming vrikshasana_sharkaboi3.jpg
Renaming vrikshasana_sharkaboi30.jpg
Renaming vrikshasana_sharkaboi300.jpg
Renaming vrikshasana_sharkaboi301.jpg
Renaming vrikshasana_sharkaboi302.jpg
Renaming vriksha

### Cleaning to (h, w, 3) format

In [75]:
path = r".\data\v2"
fols = os.listdir(path)
for f in fols:
    fil = os.listdir(os.path.join(path, f))
    for file in fil:
        fpath = os.path.join(path, f, file)
        img = plt.imread(fpath)
        if not (len(img.shape) == 3 and img.shape[2] == 3):
            print(fpath)
            
print("Done")

Done


In [23]:
visualizeDataSet()

Dataset frequency distribution
adho_mukha_shvanasana : 435
bhujangasana : 668
bidalasana : 315
phalakasana : 145
ustrasana : 66
utkatasana : 226
utkata_konasana : 209
vasisthasana : 148
virabhadrasana_i : 221
virabhadrasana_ii : 389
vrikshasana : 321

Image type frequency distribution
.jpg : 3100
.jpeg : 43



## Pre-process

In [41]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

classFolders = getClasses()
filepaths = []
labels = []

for className in classFolders:
    class_path = getPathOfClass(className)
    files = os.listdir(class_path)
    for f in files:
        fpath = os.path.join(class_path, f)
        filepaths.append(fpath)
        labels.append(className)

Fseries=pd.Series(filepaths, name='filepaths')
Lseries=pd.Series(labels, name='labels')
df=pd.concat([Fseries, Lseries], axis=1)
trsplit = 0.8
vsplit = 0.1
# split df into train_df and test_df 
dsplit=vsplit/(1-trsplit)
strat=df['labels']    
train_df, dummy_df = train_test_split(df, train_size=trsplit, shuffle=True, random_state=123, stratify=strat)
strat=dummy_df['labels']
valid_df, test_df = train_test_split(dummy_df, train_size=dsplit, shuffle=True, random_state=123, stratify=strat)
print('train_df length: ', len(train_df), '  test_df length: ',len(test_df), '  valid_df length: ', len(valid_df), ' total length: ', len(filepaths))
print(train_df['labels'].value_counts())

train_df length:  2497   test_df length:  313   valid_df length:  312  total length:  3122
bhujangasana             528
adho_mukha_shvanasana    349
virabhadrasana_ii        311
bidalasana               252
vrikshasana              250
utkatasana               178
virabhadrasana_i         175
utkata_konasana          167
vasisthasana             118
phalakasana              116
ustrasana                 53
Name: labels, dtype: int64


## Augmentation

In [43]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import shutil

def trim (df, max_size, min_size, column):
    df=df.copy()
    sample_list=[] 
    groups=df.groupby(column)
    for label in df[column].unique():        
        group=groups.get_group(label)
        sample_count=len(group)         
        if sample_count> max_size :
            samples=group.sample(max_size, replace=False, weights=None, random_state=123, axis=0).reset_index(drop=True)
            sample_list.append(samples)
        elif sample_count>= min_size:
            sample_list.append(group)
    df=pd.concat(sample_list, axis=0).reset_index(drop=True)
    balance=list(df[column].value_counts())
    print (balance)
    return df

def balance(train_df,max_samples, min_samples, column, working_dir, image_size):
    train_df=train_df.copy()
    train_df=trim (train_df, max_samples, min_samples, column)    
    # make directories to store augmented images
    aug_dir=os.path.join(working_dir, 'aug')
    if os.path.isdir(aug_dir):
        shutil.rmtree(aug_dir)
    os.mkdir(aug_dir)
    for label in train_df['labels'].unique():    
        dir_path=os.path.join(aug_dir,label)    
        os.mkdir(dir_path)
    # create and store the augmented images  
    total=0
    gen=ImageDataGenerator(horizontal_flip=True,  rotation_range=20, width_shift_range=.2,
                                  height_shift_range=.2, zoom_range=.2)
    groups=train_df.groupby('labels') # group by class
    for label in train_df['labels'].unique():  # for every class               
        group=groups.get_group(label)  # a dataframe holding only rows with the specified label 
        sample_count=len(group)   # determine how many samples there are in this class  
        if sample_count< max_samples: # if the class has less than target number of images
            aug_img_count=0
            delta=max_samples-sample_count  # number of augmented images to create
            target_dir=os.path.join(aug_dir, label)  # define where to write the images    
            aug_gen=gen.flow_from_dataframe( group,  x_col='filepaths', y_col=None, target_size=image_size,
                                            class_mode=None, batch_size=1, shuffle=False, 
                                            save_to_dir=target_dir, save_prefix='aug-', color_mode='rgb',
                                            save_format='jpg')
            while aug_img_count<delta:
                images=next(aug_gen)            
                aug_img_count += len(images)
            total +=aug_img_count
    print('Total Augmented images created= ', total)
    # create aug_df and merge with train_df to create composite training set ndf
    if total>0:
        aug_fpaths=[]
        aug_labels=[]
        classlist=os.listdir(aug_dir)
        for klass in classlist:
            classpath=os.path.join(aug_dir, klass)     
            flist=os.listdir(classpath)    
            for f in flist:        
                fpath=os.path.join(classpath,f)         
                aug_fpaths.append(fpath)
                aug_labels.append(klass)
        Fseries=pd.Series(aug_fpaths, name='filepaths')
        Lseries=pd.Series(aug_labels, name='labels')
        aug_df=pd.concat([Fseries, Lseries], axis=1)
        ndf=pd.concat([train_df,aug_df], axis=0).reset_index(drop=True)
    else:
        ndf=train_df
    print (list(ndf['labels'].value_counts()) )
    return ndf 

max_samples= 250
min_samples=0
column='labels'
working_dir = r'./'
img_size=(256, 256)
ndf=balance(train_df, max_samples, min_samples, column, working_dir, img_size)

[250, 250, 250, 250, 250, 178, 175, 167, 118, 116, 53]
Found 118 validated image filenames.
Found 178 validated image filenames.
Found 175 validated image filenames.
Found 167 validated image filenames.
Found 116 validated image filenames.
Found 53 validated image filenames.
Total Augmented images created=  693
[250, 250, 250, 250, 250, 250, 250, 250, 250, 250, 250]


### Save detected poses of training set for debugging

In [71]:
import matplotlib.pyplot as plt
import cv2

toDir = r".\intermediates\v" + str(version)

EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

# Draw Keypoints
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))

    for kp in shaped:
        ky, kx, k_conf = kp

        if k_conf > confidence_threshold:
            cv2.circle(frame, (round(kx),round(ky)), 4, (255,0,0), -1)


def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape 
    shaped = np.squeeze(np.multiply(keypoints,[y,x,1]))

    for edge,color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]

        if (c1 > confidence_threshold) & (c2 > confidence_threshold):
            cv2.line(frame, (round(x1),round(y1)), (round(x2),round(y2)), (0,255,0), 2)
                
def saveImagesWithKeypoints(image_path, keypoint_with_scores, file_name):
    image_path = os.path.abspath(image_path)
    try:
        image = plt.imread(image_path)
        image = image.copy()
        draw_connections(image, keypoint_with_scores, EDGES, 0.4)
        draw_keypoints(image, keypoints_with_scores, 0.4)
        p = Path(image_path)
        fileName = p.name
        fp = os.path.join(toDir, fileName)
        fp = r""+os.path.abspath(fp)
        if not cv2.imwrite(fp, image):
            raise Exception("image not saved to "+fp)
        print("Saved to "+ fp)
    except (Exception) as e:
        print("Failed to write image for " + image_path)
        raise e

## Running pre-trained detection model

In [72]:
import tensorflow as tf
from matplotlib.collections import LineCollection
import tqdm

modelPath = ".\models\movenet_lightning.tflite"
imageSize = 192

count = 0
for i, value in train_df.iterrows():
    image_path = value['filepaths']
    image = tf.io.read_file(image_path)
    image = tf.compat.v1.image.decode_jpeg(image)
    image = tf.image.resize_with_pad(image, imageSize, imageSize)
    image = tf.expand_dims(image, axis=0)
    interpreter = tf.lite.Interpreter(model_path=modelPath)
    interpreter.allocate_tensors()
    # TF Lite format expects tensor type of uint8.
    input_image = tf.cast(image, dtype=tf.uint8)
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    print("Detecting in "+ image_path)
    interpreter.set_tensor(input_details[0]['index'], input_image.numpy())
    interpreter.invoke()
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    saveImagesWithKeypoints(image_path, keypoints_with_scores, str(count) +".jpg")
    count+=1

Detecting in .\data\v2\vasisthasana\00000064.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\0.jpg
Detecting in .\data\v2\utkatasana\326.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\1.jpg
Detecting in .\data\v2\bidalasana\bidalasana_279.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\2.jpg
Detecting in .\data\v2\bidalasana\bidalasana_48.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\3.jpg
Detecting in .\data\v2\bhujangasana\1_50.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\4.jpg
Detecting in .\data\v2\bidalasana\bidalasana_8.jpeg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\5.jpg
Detecting in .\data\v2\adho_mukha_shvanasana\adho_mukha_shvanasana_346.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\6.jpg
Detecting in .\data\v2\bhujangasana\1_221.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\7.jpg
Detecting 

Detecting in .\data\v2\adho_mukha_shvanasana\adho_mukha_shvanasana_360.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\65.jpg
Detecting in .\data\v2\virabhadrasana_ii\00000086.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\66.jpg
Detecting in .\data\v2\virabhadrasana_ii\dwarrior_115.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\67.jpg
Detecting in .\data\v2\vasisthasana\00000090.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\68.jpg
Detecting in .\data\v2\bhujangasana\2_124.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\69.jpg
Detecting in .\data\v2\virabhadrasana_ii\00000066.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\70.jpg
Detecting in .\data\v2\virabhadrasana_ii\00000309.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\71.jpg
Detecting in .\data\v2\virabhadrasana_i\1_28.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\i

Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\131.jpg
Detecting in .\data\v2\virabhadrasana_i\1_89.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\132.jpg
Detecting in .\data\v2\vasisthasana\00000223.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\133.jpg
Detecting in .\data\v2\virabhadrasana_ii\00000221.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\134.jpg
Detecting in .\data\v2\utkatasana\13. utkatasana.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\135.jpg
Detecting in .\data\v2\bhujangasana\3_33.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\136.jpg
Detecting in .\data\v2\adho_mukha_shvanasana\adho_mukha_shvanasana_107.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\137.jpg
Detecting in .\data\v2\bidalasana\bidalasana_18.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\138.jpg
Detecting in .\data\v2\utkatasa

Detecting in .\data\v2\adho_mukha_shvanasana\adho_mukha_shvanasana_129.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\199.jpg
Detecting in .\data\v2\adho_mukha_shvanasana\adho_mukha_shvanasana_344.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\200.jpg
Detecting in .\data\v2\adho_mukha_shvanasana\adho_mukha_shvanasana_179.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\201.jpg
Detecting in .\data\v2\virabhadrasana_ii\00000095.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\202.jpg
Detecting in .\data\v2\virabhadrasana_ii\51-0.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\203.jpg
Detecting in .\data\v2\adho_mukha_shvanasana\adho_mukha_shvanasana_408.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\204.jpg
Detecting in .\data\v2\virabhadrasana_i\23-0.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\205.jpg
Detecting in .\data\v2\virabh

Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\264.jpg
Detecting in .\data\v2\adho_mukha_shvanasana\adho_mukha_shvanasana_248.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\265.jpg
Detecting in .\data\v2\virabhadrasana_ii\dwarrior_117.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\266.jpg
Detecting in .\data\v2\utkata_konasana\goddess (59).jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\267.jpg
Detecting in .\data\v2\virabhadrasana_ii\00000023.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\268.jpg
Detecting in .\data\v2\vasisthasana\30-0.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\269.jpg
Detecting in .\data\v2\bhujangasana\1_530.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\270.jpg
Detecting in .\data\v2\bidalasana\bidalasana_46.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\271.jpg
Detecting in .\data\v2

...

Detecting in .\data\v2\utkatasana\9. untitled-design1 copy.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\597.jpg
Detecting in .\data\v2\adho_mukha_shvanasana\adho_mukha_shvanasana_105.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\598.jpg
Detecting in .\data\v2\utkatasana\470.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\599.jpg
Detecting in .\data\v2\vrikshasana\File84.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\600.jpg
Detecting in .\data\v2\virabhadrasana_ii\27-0.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\601.jpg
Detecting in .\data\v2\adho_mukha_shvanasana\adho_mukha_shvanasana_275.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\602.jpg
Detecting in .\data\v2\bhujangasana\2_400.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\603.jpg
Detecting in .\data\v2\vasisthasana\48-1.jpg
Saved to A:\Projects\CollegeProjects\Yog

Detecting in .\data\v2\vrikshasana\00000055.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\664.jpg
Detecting in .\data\v2\utkata_konasana\00000053.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\665.jpg
Detecting in .\data\v2\bhujangasana\1_314.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\666.jpg
Detecting in .\data\v2\adho_mukha_shvanasana\adho_mukha_shvanasana_81.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\667.jpg
Detecting in .\data\v2\bhujangasana\3_68.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\668.jpg
Detecting in .\data\v2\virabhadrasana_i\1_550.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\669.jpg
Detecting in .\data\v2\utkatasana\47. utkatasana-steps-benefits-vidhi-fayde-in-hindi.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\670.jpg
Detecting in .\data\v2\vrikshasana\00000079.jpg
Saved to A:\Projects\CollegePr

Detecting in .\data\v2\utkatasana\8. 3+big+ideas+for+improving+your+utkatasana.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\728.jpg
Detecting in .\data\v2\utkata_konasana\goddess (49).jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\729.jpg
Detecting in .\data\v2\adho_mukha_shvanasana\adho_mukha_shvanasana_212.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\730.jpg
Detecting in .\data\v2\utkata_konasana\00000130.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\731.jpg
Detecting in .\data\v2\adho_mukha_shvanasana\adho_mukha_shvanasana_295.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\732.jpg
Detecting in .\data\v2\utkata_konasana\00000237.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\733.jpg
Detecting in .\data\v2\bhujangasana\2_169.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\734.jpg
Detecting in .\data\v2\utkatasana\408.jpg
S

Detecting in .\data\v2\bhujangasana\3_54.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\792.jpg
Detecting in .\data\v2\utkatasana\313.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\793.jpg
Detecting in .\data\v2\virabhadrasana_ii\33-0.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\794.jpg
Detecting in .\data\v2\utkata_konasana\00000313.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\795.jpg
Detecting in .\data\v2\adho_mukha_shvanasana\adho_mukha_shvanasana_264.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\796.jpg
Detecting in .\data\v2\bhujangasana\1_286.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\797.jpg
Detecting in .\data\v2\vasisthasana\24-0.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\798.jpg
Detecting in .\data\v2\utkatasana\413.jpg
Saved to A:\Projects\CollegeProjects\YogaPartner\intermediates\v2\799.jpg
Detecting i

ValueError: Cannot set tensor: Dimension mismatch. Got 4 but expected 3 for dimension 3 of input 0.

## Train classifier model

In [ ]:
X, y, class_names = load_csv('train_data.csv')
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15)
X_test, y_test, _ = load_csv('test_data.csv')


processed_X_train = preprocess_data(X_train)
processed_X_val =  preprocess_data(X_val)
processed_X_test = preprocess_data(X_test)

inputs = tf.keras.Input(shape=(34))
layer = keras.layers.Dense(128, activation=tf.nn.relu6)(inputs)
layer = keras.layers.Dropout(0.5)(layer)
layer = keras.layers.Dense(64, activation=tf.nn.relu6)(layer)
layer = keras.layers.Dropout(0.5)(layer)
outputs = keras.layers.Dense(len(class_names), activation="softmax")(layer)

model = keras.Model(inputs, outputs)


model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Add a checkpoint callback to store the checkpoint that has the highest
# validation accuracy.
checkpoint_path = "weights.best.hdf5"
checkpoint = keras.callbacks.ModelCheckpoint(checkpoint_path,
                             monitor='val_accuracy',
                             verbose=1,
                             save_best_only=True,
                             mode='max')
earlystopping = keras.callbacks.EarlyStopping(monitor='val_accuracy', 
                                              patience=20)

# Start training
print('--------------TRAINING----------------')
history = model.fit(processed_X_train, y_train,
                    epochs=200,
                    batch_size=16,
                    validation_data=(processed_X_val, y_val),
                    callbacks=[checkpoint, earlystopping])